In [5]:
!pwd

/Users/sushmitha047/Documents/RealWorldML-Pau/taxi_demand_predictor


In [6]:
import os
os.chdir("..")

In [25]:
%reload_ext autoreload
%autoreload 2

In [26]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / "final_data.parquet")
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89300,3.0,0.0,2.0,3.0,2.0,3.0,13.0,8.0,9.0,9.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
89301,6.0,4.0,0.0,0.0,2.0,0.0,14.0,7.0,8.0,4.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
89302,7.0,2.0,3.0,4.0,7.0,4.0,10.0,9.0,7.0,11.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
89303,6.0,5.0,4.0,3.0,0.0,3.0,11.0,12.0,9.0,10.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [28]:
df.dtypes

rides_previous_672_hour                float32
rides_previous_671_hour                float32
rides_previous_670_hour                float32
rides_previous_669_hour                float32
rides_previous_668_hour                float32
                                  ...         
rides_previous_2_hour                  float32
rides_previous_1_hour                  float32
pickup_hour                datetime64[ns, UTC]
pickup_location_id                       int64
target_rides_next_hour                 float32
Length: 675, dtype: object

In [ ]:
from src.data_split import train_test_split
from datetime import datetime
import pytz

cutoff_date = datetime(2022, 8, 1, 0, 0, 0)

X_train, y_train, X_test, y_test = train_test_split(
    df,
    cutoff_date=cutoff_date,
    target_column_name='target_rides_next_hour'
)

TypeError: Invalid comparison between dtype=datetime64[ns, UTC] and datetime

In [6]:
print(f"{X_train.shape=}")
print(f"{y_train.shape=}")
print(f"{X_test.shape=}")
print(f"{y_test.shape=}")

X_train.shape=(48208, 674)
y_train.shape=(48208,)
X_test.shape=(40086, 674)
y_test.shape=(40086,)


## Creating baseline model 1: predicting previous 1 hour rides

#### Inroder to predict taxi demand for the next hour, we'll look at what was the demand for previous hour

In [7]:
import numpy as np

class BaselineModelPreviousHour:
    """
    Prediction = actual demand observed in the last hour
    """

    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f'rides_previous_1_hour']

In [8]:
model = BaselineModelPreviousHour() # created model object
predictions = model.predict(X_test)
predictions

0         0.0
1         2.0
2         3.0
3         4.0
4        15.0
         ... 
40081     0.0
40082     0.0
40083     0.0
40084     0.0
40085     0.0
Name: rides_previous_1_hour, Length: 40086, dtype: float32

In [9]:
from sklearn.metrics import mean_absolute_error

test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae=:.4f}")


test_mae=6.2000


## Creating baseline model 2: predicting previous 1 week rides

In [10]:
class BaselineModelPreviousWeek:
    """
    Prediction = actual demand observed at t - 7 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame) -> np.array:
        """"""
        return X_test[f"rides_previous_{7*24}_hour"]

In [11]:
model = BaselineModelPreviousWeek()
predictions = model.predict(X_test)
predictions

0        2.0
1        2.0
2        2.0
3        1.0
4        7.0
        ... 
40081    0.0
40082    0.0
40083    0.0
40084    0.0
40085    0.0
Name: rides_previous_168_hour, Length: 40086, dtype: float32

In [12]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae=:.4f}")

test_mae=3.8745


## Creating baseline model 3: predicting average of last 4 weeks

In [13]:
class BaselineModelLast4Weeks:
    """
    Prediction = actual demand observed at t - 7 days, t - 14 days, t - 21 days, t - 28 days
    """
    def fit(self, X_train: pd.DataFrame, y_train: pd.Series):
        pass

    def predict(self, X_test: pd.DataFrame):
        # 0.25 - for the average of 4
        return 0.25*(
            X_test[f"rides_previous_{7*24}_hour"] + \
            X_test[f"rides_previous_{2*7*24}_hour"] + \
            X_test[f"rides_previous_{3*7*24}_hour"] + \
            X_test[f"rides_previous_{4*7*24}_hour"]
        )

In [14]:
model = BaselineModelLast4Weeks()
predictions = model.predict(X_test)
predictions

0        3.00
1        2.50
2        2.25
3        2.25
4        9.00
         ... 
40081    0.00
40082    0.00
40083    0.00
40084    0.00
40085    0.00
Length: 40086, dtype: float32

In [15]:
test_mae = mean_absolute_error(y_test, predictions)
print(f"{test_mae=:.4f}")

test_mae=3.3684
